<a href="https://colab.research.google.com/github/0-lingual/DBGAPS_StockPrice/blob/main/DBGAPS_dataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pandas yfinance schedule

In [21]:
import FinanceDataReader as fdr
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta
import threading
import os

# ETF 목록
etf_list = {
    'KODEX 200': '069500',
    'TIGER 코스닥150': '232080',
    'TIGER 미국S&P500선물(H)': '143850',
    'TIGER 유로스탁스50(합성,H)': '195930',
    'ACE 일본Nikkei225(H)': '241180',
    'TIGER 차이나CSI300': '192090',
    'KOSEF 국고채10년': '148070',
    'KBSTAR 중기우량회사채': '136340',
    'TIGER 단기선진하이일드(합성,H)': '182490',
    'KODEX 골드선물(H)': '132030',
    'TIGER 원유선물Enhanced(H)': '130680',
    'KODEX 인버스': '114800',
    'KOSEF 미국달러선물': '138230',
    'KOSEF 미국달러선물인버스': '139660',
    'KOSEF 단기자금': '130730'
}

# 첫 실행 시 초기 데이터 로드
def initial_load():
    start_date = '2014-01-01'
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    data = {'날짜': [], '종목명': [], '종가': []}

    for name, code in etf_list.items():
        df = fdr.DataReader(code, start=start_date, end=end_date)
        for date, row in df.iterrows():
            data['날짜'].append(date.strftime('%Y-%m-%d'))
            data['종목명'].append(name)
            data['종가'].append(row['Close'])

    df_initial = pd.DataFrame(data)
    df_initial.to_csv('etf_prices.csv', mode='w', header=True, index=False)
    print("Initial data load complete.")

# 매일 데이터 업데이트 함수
def update_etf_data():
    today = datetime.now().strftime('%Y-%m-%d')
    data = {'날짜': [], '종목명': [], '종가': []}

    for name, code in etf_list.items():
        df = fdr.DataReader(code, start=today, end=today)
        if not df.empty:
            closing_price = df['Close'].iloc[-1]
            data['날짜'].append(today)
            data['종목명'].append(name)
            data['종가'].append(closing_price)

    df_daily = pd.DataFrame(data)
    df_daily.to_csv('etf_prices.csv', mode='a', header=False, index=False)
    print(f"Data updated for {today}")

# 스케줄러를 별도의 스레드에서 실행
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)

# 매일 특정 시간에 데이터 업데이트
schedule.every().day.at("17:00").do(update_etf_data)

# 첫 데이터 로드 실행
if not os.path.exists('etf_prices.csv'):
    initial_load()

# 첫 데이터 업데이트 즉시 실행
update_etf_data()

# 스케줄러 스레드 시작
scheduler_thread = threading.Thread(target=run_schedule)
scheduler_thread.daemon = True
scheduler_thread.start()

# 메인 프로그램 실행
print("Scheduler started. Waiting for the scheduled time...")

Initial data load complete.
Data updated for 2024-06-17
Scheduler started. Waiting for the scheduled time...
